<a href="https://colab.research.google.com/github/vishesh711/kaggle_competition_Poisonous_Mushrooms_RandomForest/blob/main/kaggle_competition_Poisonous_Mushrooms_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import matthews_corrcoef
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [3]:
!kaggle competitions download -c playground-series-s4e8

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [4]:
train=pd.read_csv("/content/train.csv", index_col=0)
test=pd.read_csv("/content/test.csv", index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '/content/train.csv'

In [ ]:
print(f'Training Set\nRows: {train.shape[0]} | Cols: {train.shape[1]}\n')

print(f'Testing Set\nRows: {test.shape[0]} | Cols: {test.shape[1]}')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
train.describe().T.round(2)

In [ ]:
# Check for missing values
train_na = (train.isna()
                    .sum()
                    .reset_index()
                    .sort_values(0, ascending=False)
                ).rename(columns={'index':'col',0:'na'})

train_na['percentage_na'] = (train_na['na'] / len(train)).round(2)

train_na.style.background_gradient()

In [ ]:
train[train.duplicated()]

In [ ]:
cols_drop = ['veil-type', 'spore-print-color', 'stem-root', 'veil-color', 'stem-surface']

In [ ]:
train.drop(columns=cols_drop, inplace=True)
test.drop(columns=cols_drop, inplace=True)

In [ ]:
categorical_var = train.select_dtypes('object').columns
continous_var = train.select_dtypes('float').columns

In [ ]:
# Check labels
pd.set_option('display.max_rows', None)

for col in train[categorical_var].columns:
    print(train[col].value_counts())

pd.set_option('display.max_rows', 5)

In [ ]:
# Create a list containing the alphabet letters
alhpabet = list(map(chr,range(97,123)))

In [ ]:
# Fixing some values
train[categorical_var] = train[categorical_var].map(lambda x: 'Unkwnown' if x not in alhpabet else x)
test[categorical_var[1:]] = test[categorical_var[1:]].map(lambda x: 'Unkwnown' if x not in alhpabet else x)

In [ ]:
train['count'] = 1

In [ ]:
palette = ['green', 'red']

for num_var in continous_var:

    fig, ax = plt.subplots(1, 2, figsize=(12, 3))

    # Histograms
    sns.histplot(data=train,
                      x=num_var,
                      bins = 30,
                      hue = 'class',
                      palette=palette,
                      #kde=True,
                      alpha=0.3,
                      ax=ax[0]
                )
    # Mean vertical line
    ax[0].axvline(np.mean(train[num_var]), color="red")

    # Boxplots
    sns.boxplot(data=train,
                     y=num_var,
                     hue = 'class',
                     palette=palette,
                     #showfliers=False,
                     ax=ax[1]
                   );

In [ ]:
for col in train[categorical_var].columns:
    if col != 'class':
        plt.figure(figsize=(15,5,))
        sns.histplot(data=train[[col,'class','count']].groupby([col,'class']).sum().reset_index(),
                        x=col,
                        weights='count',
                        hue='class',
                        multiple='stack',
                        palette=palette,
                        edgecolor='white',
                        shrink=0.8
                    );

In [ ]:
# Replace missing continous var with the respective mean value
train['cap-diameter'] = train['cap-diameter'].replace(np.nan,train['cap-diameter'].mean())
train['stem-height'] = train['stem-height'].replace(np.nan,train['stem-height'].mean())

test['cap-diameter'] = test['cap-diameter'].replace(np.nan,test['cap-diameter'].mean())
test['stem-height'] = test['stem-height'].replace(np.nan,test['stem-height'].mean())

In [ ]:
# Encode the categorical variables
enc = LabelEncoder()

for var in categorical_var:
    train[var] = enc.fit_transform(train[var])


for var in categorical_var[1:]:
    test[var] = enc.fit_transform(test[var])

In [ ]:
# Crate independent and dependent varaibles
X = train.drop(columns=['count','class'])
y = train['class']

In [ ]:
# Split the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
rf = RandomForestClassifier(random_state=0)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
matthews_corrcoef(y_test, y_pred)

SUB


In [ ]:
y_pred = rf.predict(test)

In [ ]:
test_predictions = np.where(y_pred == 0, 'e', 'p')

In [ ]:
submission_df = pd.DataFrame({'id': test.index,
                              'Target':test_predictions})

In [ ]:
submission_df.to_csv('submission.csv', index=False)
submission_df.head()